In [104]:
import os
import re
import pandas as pd
from pybaseball import statcast_batter, statcast_pitcher, playerid_lookup, pitching_stats_range, batting_stats_range, schedule_and_record, team_game_logs, pybaseball
from datetime import timedelta, datetime
import statsapi
import pprint

In [19]:
# Function to create a directory for player data if it doesn't exist
def create_player_dir(player_type):
    if not os.path.exists(player_type):
        os.makedirs(player_type)

In [20]:
# Function to store player data in a CSV file
def store_player_data(player_id, data, player_type):
    create_player_dir(player_type)
    file_path = f"{player_type}/{player_id}.csv"
    data.to_csv(file_path, index=False)

In [49]:
# Function to get all player IDs for a season
def get_all_player_ids(season):
    # You can replace this with a CSV file containing player IDs for each team

    fname = '~\Development\MLB-Analytics\processed_pitching_data.csv'
    

    #roster_url = f'https://someurl.com/mlb_rosters_{season}.csv'  # Placeholder URL
    rosters = pd.read_csv(fname)
    #return rosters['mlbID'].unique()
    ohtani_id = [660271]
    return ohtani_id



<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\kesse\AppData\Local\Temp\ipykernel_15720\3098783967.py:5: SyntaxWarning: invalid escape sequence '\D'
  fname = '~\Development\MLB-Analytics\processed_pitching_data.csv'


In [50]:
# Function to fetch and store batter data
def fetch_and_store_batter_data(start_date, end_date, player_ids):
    for player_id in player_ids:
        try:
            player_data = statcast_batter(start_date, end_date, player_id=player_id)
            if not player_data.empty:
                store_player_data(player_id, player_data, 'batters')
        except Exception as e:
            print(f"Error fetching data for batter {player_id}: {e}")


In [51]:
# Function to fetch and store pitcher data
def fetch_and_store_pitcher_data(start_date, end_date, player_ids):
    for player_id in player_ids:
        try:
            player_data = statcast_pitcher(start_date, end_date, player_id=player_id)
            if not player_data.empty:
                store_player_data(player_id, player_data, 'pitchers')
        except Exception as e:
            print(f"Error fetching data for pitcher {player_id}: {e}")


In [65]:
start_date = '2021-04-01'

today = datetime.now()
end_date = today.strftime('%Y-%m-%d')

season = today.year

one_hour_before_game = today - timedelta(hours=1)

In [63]:
# Fetch player IDs
player_ids = get_all_player_ids(season)

In [64]:
# Fetch and store data
fetch_and_store_batter_data(start_date, end_date, player_ids)
fetch_and_store_pitcher_data(start_date, end_date, player_ids)

Gathering Player Data
Gathering Player Data


## Testing

In [84]:
team = 'SD'
season = 2024
day = today.day-1
month_str = today.strftime('%B') # Full month name

month_day_str = f'{month_str} + +{day}'
day_string = re.compile(rf'{month_str}\s+{day}')

In [85]:
game_log = schedule_and_record(season, team)
game_log.loc[game_log.Date.str.contains(day_string), :]

http://www.baseball-reference.com/teams/SD/2024-schedule-scores.shtml


c:\Users\kesse\Development\MLB-Analytics\MLBpy\Lib\site-packages\pybaseball\team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
53,"Thursday, May 23",SDP,@,CIN,W,6.0,4.0,10.0,27-26,2.0,6.5,Estrada,Moll,Suarez,3:15,D,16479.0,.99,2.0,None


In [93]:
sched = statsapi.schedule(start_date, end_date)
print(sched)

[{'game_id': 746475, 'game_datetime': '2024-05-24T22:40:00Z', 'game_date': '2024-05-24', 'game_type': 'R', 'status': 'Scheduled', 'away_name': 'Toronto Blue Jays', 'home_name': 'Detroit Tigers', 'away_id': 141, 'home_id': 116, 'doubleheader': 'N', 'game_num': 1, 'home_probable_pitcher': 'Matt Manning', 'away_probable_pitcher': 'Alek Manoah', 'home_pitcher_note': '', 'away_pitcher_note': '', 'away_score': '0', 'home_score': '0', 'current_inning': '', 'inning_state': '', 'venue_id': 2394, 'venue_name': 'Comerica Park', 'national_broadcasts': [], 'series_status': None, 'summary': '2024-05-24 - Toronto Blue Jays @ Detroit Tigers (Scheduled)'}, {'game_id': 745501, 'game_datetime': '2024-05-24T22:40:00Z', 'game_date': '2024-05-24', 'game_type': 'R', 'status': 'Scheduled', 'away_name': 'Atlanta Braves', 'home_name': 'Pittsburgh Pirates', 'away_id': 144, 'home_id': 134, 'doubleheader': 'N', 'game_num': 1, 'home_probable_pitcher': 'Bailey Falter', 'away_probable_pitcher': '', 'home_pitcher_note

In [94]:
player = statsapi.lookup_player('ohtani,')
print(player[0]['id']) #assume only 1 record returned for demo purposes

660271


In [114]:
gamePk = 632190
data = statsapi.boxscore_data(gamePk)

print(data)


{'gameId': '2021/04/13/bosmlb-minmlb-1', 'teamInfo': {'away': {'id': 111, 'abbreviation': 'BOS', 'teamName': 'Red Sox', 'shortName': 'Boston'}, 'home': {'id': 142, 'abbreviation': 'MIN', 'teamName': 'Twins', 'shortName': 'Minnesota'}}, 'playerInfo': {'ID593958': {'id': 593958, 'fullName': 'Eduardo Rodriguez', 'boxscoreName': 'Rodriguez, Ed'}, 'ID517008': {'id': 517008, 'fullName': 'Alex Colomé', 'boxscoreName': 'Colomé'}, 'ID664247': {'id': 664247, 'fullName': 'Kyle Garlick', 'boxscoreName': 'Garlick'}, 'ID592743': {'id': 592743, 'fullName': 'Andrelton Simmons', 'boxscoreName': 'Simmons'}, 'ID592669': {'id': 592669, 'fullName': 'Hunter Renfroe', 'boxscoreName': 'Renfroe'}, 'ID663399': {'id': 663399, 'fullName': 'Brandon Waddell', 'boxscoreName': 'Waddell'}, 'ID571771': {'id': 571771, 'fullName': 'Enrique Hernández', 'boxscoreName': 'Hernández, K'}, 'ID657077': {'id': 657077, 'fullName': 'Alex Verdugo', 'boxscoreName': 'Verdugo'}, 'ID608700': {'id': 608700, 'fullName': 'Kevin Plawecki',

In [111]:
def get_all_game_pks_since_2021(start_year=2021):
    game_pks = []
    
    # Iterate through each year starting from 2021 to the current year
    for year in range(start_year, 2024):
        # Get the schedule for the year
        schedule = statsapi.schedule(start_date=f'{year}-01-01', end_date=f'{year}-12-31')
        
        # Extract gamePk IDs from the schedule
        for game in schedule:
            game_pks.append(game['game_id'])
    
    # Sort the gamePk IDs (if needed, they should already be in order)
    game_pks.sort()

    return game_pks

# Example usage
game_pks = get_all_game_pks_since_2021()
print(game_pks)

len(game_pks)

[632169, 632170, 632188, 632189, 632190, 632191, 632192, 632192, 632193, 632194, 632195, 632196, 632197, 632198, 632199, 632200, 632201, 632202, 632203, 632204, 632205, 632206, 632207, 632208, 632208, 632209, 632210, 632211, 632212, 632213, 632214, 632215, 632216, 632217, 632218, 632219, 632220, 632221, 632222, 632223, 632223, 632224, 632224, 632225, 632226, 632226, 632227, 632228, 632229, 632230, 632231, 632232, 632233, 632234, 632235, 632236, 632237, 632238, 632239, 632240, 632241, 632242, 632243, 632244, 632245, 632246, 632247, 632248, 632249, 632250, 632251, 632252, 632253, 632254, 632255, 632256, 632257, 632258, 632259, 632260, 632261, 632262, 632263, 632264, 632265, 632266, 632267, 632268, 632269, 632270, 632271, 632272, 632273, 632274, 632275, 632276, 632277, 632278, 632279, 632280, 632281, 632282, 632283, 632284, 632285, 632286, 632287, 632288, 632289, 632290, 632291, 632292, 632293, 632294, 632295, 632296, 632297, 632298, 632299, 632300, 632301, 632302, 632303, 632304, 632305,